# Le jeu Palworld et analyse des données

## Présentation du jeu Palworld

Palworld est un jeu de survie en monde ouvert développé par Pocket Pair, sorti en janvier 2024. Ce jeu combine des éléments de survie, de crafting, de collection de créatures et de construction de base. Il se déroule dans un monde ouvert peuplé de créatures appelées "Pals" que les joueurs peuvent capturer, élever et utiliser pour diverses tâches.

Les aspects clés du jeu incluent:

| Aspect |
| - |
| La capture et la collection de Pals (similaire à Pokémon) |
| L'utilisation des Pals pour combattre |
| L'exploitation des Pals pour des tâches comme la construction, l'agriculture, et le travail en usine |
| La construction et le développement d'un campement |
| L'exploration d'un monde ouvert avec différentes zones et biomes |

Le jeu a connu un immense succès dès sa sortie en raison de son concept innovant mêlant des mécaniques de jeux populaires comme Pokémon et Minecraft, mais avec une approche plus mature et parfois controversée.


## 📁 Analyse des Fichiers CSV et leur Rôle dans le Jeu

### Vue d'ensemble des Données Palworld

Sur la base des descriptions fournies, voici une analyse de chaque fichier de données et son importance dans le jeu :

### 1.  Palworld_Data--Palu combat attribute table.csv

**Contenu :**
- Statistiques de base (santé, attaque, défense, vitesse)
- Types élémentaires (Feu, Eau, Électricité, etc.)
- Compétences de combat spécifiques

**Rôle dans le jeu :** Ces données déterminent l'efficacité d'un Pal au combat. Les joueurs utilisent ces informations pour construire des équipes équilibrées et stratégiques pour les combats contre d'autres Pals ou les boss.

### 2.  Palworld_Data--Palu refresh level.csv

**Contenu :** Informations sur les niveaux auxquels les Pals apparaissent dans différentes zones du jeu.

**Rôle dans le jeu :** Cette information est cruciale pour les joueurs qui cherchent à capturer des Pals spécifiques. Elle permet de savoir où trouver certains Pals et à quel niveau ils apparaîtront, ce qui affecte leur difficulté de capture et leurs statistiques initiales.

### 3. Palworld_Data-Palu Job Skills Table.csv

**Contenu :** Compétences de travail que possèdent les différents Pals :
- Minage
- Bûcheronnage
- Agriculture
- Construction
- Cuisine
- Etc.

**Rôle dans le jeu :** Ces compétences déterminent quelles tâches un Pal peut effectuer dans votre campement. Un Pal avec une compétence de minage élevée sera efficace pour extraire des minerais, tandis qu'un Pal avec des compétences agricoles sera utile pour cultiver des ressources alimentaires.

### 4.  Palworld_Data-Tower BOSS attribute comparison.csv

**Contenu :** Attributs des boss spéciaux qui apparaissent dans les tours du jeu.

**Rôle dans le jeu :** Les tours sont des défis endgame où les joueurs affrontent des boss puissants. Ces données aident à comprendre la puissance relative de ces boss et à préparer des stratégies adaptées pour les combattre.

### 5.  Palworld_Data-comparison of ordinary BOSS attributes.csv

**Contenu :** Attributs des boss ordinaires que l'on trouve dans le monde ouvert.

**Rôle dans le jeu :** Ces boss représentent des défis importants durant l'exploration et fournissent des récompenses précieuses. Comprendre leurs attributs permet aux joueurs de se préparer adéquatement à ces affrontements.

### 6.  Palworld_Data-hide pallu attributes.csv

**Contenu :** Informations sur des attributs cachés des Pals qui ne sont pas immédiatement visibles dans le jeu.

**Rôle dans le jeu :** Ces attributs cachés peuvent inclure des taux de croissance spécifiques, des affinités particulières, ou des capacités spéciales qui se débloquent dans certaines conditions. Ces informations sont précieuses pour les joueurs qui cherchent à optimiser leurs Pals.

---
## Implications pour l'Analyse

Cette structure de données nous permettra d'analyser :
- **Optimisation des équipes de combat** via les attributs de combat
- **Gestion efficace des ressources** via les compétences de travail
- **Stratégies de capture** via les zones d'apparition et probabilités
- **Progression du jeu** via les niveaux et raretés des Pals

## Import des bibliothèques Python de l'étude

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mariadb
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path

## Chargement des données brutes

In [41]:
# Définition du chemin vers les fichiers fournis
database_path = "../database/"
data_path = "../data/"

# Vérification de l'existence du repertoire , if not utiliser le repertoire courrant
if not os.path.exists(data_path):
    data_path = "./"

# Affichage des fichiers dispo pour la vérif
files = [f for f in os.listdir(data_path) if f.endswith('.csv')] # ici on liste tous les fichiers ou dossiers + on itère chaque élément + Filtre pour garder que les fichier .csv
print("Fichiers dispo:")
for file in files: 
    print(f"-{file}")

# Chargement des données brutes & gestion d'erreurs grâce à pd.read_csv en définissant le chemin vers les fichiers.csv 
try:
    combat_attribute = pd.read_csv(f'{data_path}Palworld_Data--Palu combat attribute table.csv')
    refresh_area = pd.read_csv(f'{data_path}Palworld_Data--Palu refresh level.csv')
    ordinary_boss = pd.read_csv(f'{data_path}Palworld_Data-comparison of ordinary BOSS attributes.csv')
    tower_boss = pd.read_csv(f'{data_path}Palworld_Data-Tower BOSS attribute comparison.csv') 
    job_skill = pd.read_csv(f'{data_path}Palworld_Data-Palu Job Skills Table.csv')
    hidden_attribute = pd.read_csv(f'{data_path}Palworld_Data-hide pallu attributes.csv')
    print("\nDonnées brutes chargées")
except FileNotFoundError as e : 
  print(f"Erreur: {e}. Vérifier les noms des fichiers et le chemin.")

Fichiers dispo:
-Palworld_Data--Palu combat attribute table.csv
-Palworld_Data--Palu refresh level.csv
-Palworld_Data-comparison of ordinary BOSS attributes.csv
-Palworld_Data-hide pallu attributes.csv
-Palworld_Data-Palu Job Skills Table.csv
-Palworld_Data-Tower BOSS attribute comparison.csv

Données brutes chargées


## Création de la fonction d'analyse exploratoires des données 

In [42]:
def explore_dataframe(df, name):
    # Cette Fonction inslus un affichage optimisé
    
    # Sauvegarde et réglage des options d'affichage
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 30)
    pd.set_option('display.width', max(1200, df.shape[1] * 20))

    print(f"\n===== Exploration de {name} =====")

    # 1. Informations de base (similaire à df.info())
    print("\n--- Informations de base ---")
    print(f"Class: {type(df)}")
    # Index
    idx = df.index
    print(f"Index: {type(idx)} | {idx}")
        # Nombre total d'entrées et de colonnes
    print(f"Entries: {len(df)} | Columns: {df.shape[1]}")
    # Détail des colonnes et non-null counts
    structure = pd.DataFrame({
        'Column': df.columns,
        'Non-Null Count': df.notnull().sum().values,
        'Dtype': df.dtypes.values
    })
    display(structure)
    # Usage mémoire
    mem_usage = df.memory_usage(deep=True).sum()
    print(f"Memory usage: {mem_usage / 1024:.2f} KB")

    # 2. Dimensions
    print(f"\nDimensions: {df.shape[0]} lignes, {df.shape[1]} colonnes")
    
    # Aperçu des premières lignes
    print("\nAperçu des premières lignes:")
    display(df.head(3))
    
    # Types de données
    print("\nTypes de données:")
    display(df.dtypes)
    
    # Statistiques descriptives
    print("\nStatistiques descriptives:")
    display(df.describe(include='all').T)
    
    # Valeurs manquantes
    print("\nValeurs manquantes:")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_data = pd.concat([missing, missing_percent], axis=1, keys=['Total', 'Pourcentage'])
    display(missing_data[missing_data['Total'] > 0])
    if missing_data['Total'].sum() == 0:
        print("Aucune valeur manquante détectée")
    
    # Doublons
    duplicates = df.duplicated().sum()
    print(f"\nNombre de lignes dupliquées: {duplicates}")
    
    # Colonnes catégorielles
    print("\nAnalyse des colonnes catégorielles:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            unique_values = df[col].unique()
            n_unique = len(unique_values)
            
            print(f"\n{col}:")
            print(f"  - {n_unique} valeurs uniques")
            
            if n_unique <= 10:
                print(f"  - Valeurs: {unique_values}")
            elif n_unique <= 20:
                value_counts = df[col].value_counts().head(10)
                print(f"  - Top 10 des valeurs les plus fréquentes:")
                for val, count in value_counts.items():
                    print(f"    • {val}: {count} occurrences")
            else:
                value_counts = df[col].value_counts()
                print(f"  - Top 5 des valeurs les plus fréquentes:")
                for val, count in value_counts.head(5).items():
                    print(f"    • {val}: {count} occurrences")
                print(f"  - Valeur la moins fréquente: {value_counts.tail(1).index[0]} ({value_counts.tail(1).values[0]} occurrence(s))")
    else:
        print("Aucune colonne catégorielle détectée")
    
    # Rétablissement des options d'affichage
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_info_columns')
    pd.reset_option('display.max_colwidth')
    pd.reset_option('display.width')

    # Séparateur
    print("\n" + "="*70)

### Analyse exploratoire des données 

In [43]:
explore_dataframe(job_skill, 'Job Skills')


===== Exploration de Job Skills =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=138, step=1)
Entries: 138 | Columns: 23


,Column,Non-Null Count,Dtype
0,ID,138,int64
1,English name,138,object
2,Chinese name,138,object
3,Volume size,138,object
4,Food intake,138,int64
5,night shift,138,bool
6,Total skills,138,int64
7,Make a fire,138,bool
8,watering,138,bool
9,planting,138,bool


Memory usage: 46.97 KB

Dimensions: 138 lignes, 23 colonnes

Aperçu des premières lignes:


,ID,English name,Chinese name,Volume size,Food intake,night shift,Total skills,Make a fire,watering,planting,generate electricity,manual,collection,logging,Mining,pharmaceutical,cool down,pasture,carry,Handling speed,ranch items,pasture minimum output,The largest ranch
0,4,Lifmunk,green leaf rat,smallest,1,False,5,False,False,True,False,True,True,True,False,True,False,False,False,NaN,NaN,NaN,NaN
1,5,Foxparks,tinder fox,smallest,2,False,1,True,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN
2,9,Rooby,Fire Deer,Small,3,False,1,True,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,138.0,NaN,NaN,NaN,57.826087,31.829361,1.0,31.25,58.5,85.75,111.0
English name,138,138,Lifmunk,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chinese name,138,137,Thunderbird,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Volume size,138,5,big,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Food intake,138.0,NaN,NaN,NaN,4.550725,2.263878,1.0,3.0,5.0,7.0,9.0
night shift,138,2,False,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total skills,138.0,NaN,NaN,NaN,4.188406,2.703204,1.0,2.0,4.0,6.0,12.0
Make a fire,138,2,False,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
watering,138,2,False,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
planting,138,2,False,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
Handling speed,82,59.42029
ranch items,125,90.57971
pasture minimum output,125,90.57971
The largest ranch,125,90.57971



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

English name:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Lifmunk: 1 occurrences
    • Foxparks: 1 occurrences
    • Rooby: 1 occurrences
    • Jolthog Cryst: 1 occurrences
    • Gumoss: 1 occurrences
  - Valeur la moins fréquente: Orserk (1 occurrence(s))

Chinese name:
  - 137 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • tinder fox: 1 occurrences
    • Fire Deer: 1 occurrences
    • ice agouti: 1 occurrences
    • Ye NiNi: 1 occurrences
  - Valeur la moins fréquente: Pollux (1 occurrence(s))

Volume size:
  - 5 valeurs uniques
  - Valeurs: ['smallest' 'Small' 'medium' 'big' 'maximum']

ranch items:
  - 11 valeurs uniques
  - Top 10 des valeurs les plus fréquentes:
    • wool: 3 occurrences
    • gold: 2 occurrences
    • Palu Ball Advanced Palu Ball Arrow Gold Coin: 1 occurrences
    • Egg: 1 occurrences
    • milk: 1 occurrences
    

In [49]:
explore_dataframe(refresh_area, 'Refresh area')


===== Exploration de Refresh area =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=136, step=1)
Entries: 136 | Columns: 8


,Column,Non-Null Count,Dtype
0,id,136,int64
1,name,136,object
2,minimum level,136,int64
3,maximum level,136,int64
4,fecondity,136,int64
5,pallu refresh type,136,object
6,night only,136,bool
7,refresh area,136,object


Memory usage: 30.92 KB

Dimensions: 136 lignes, 8 colonnes

Aperçu des premières lignes:


,id,name,minimum level,maximum level,fecondity,pallu refresh type,night only,refresh area
0,1,Mian Youyou,1,14,1470,Creeps,False,grassland
1,2,Naughty cat,1,13,1460,Creeps,False,grassland
2,3,Pipi Chicken,1,13,1500,Creeps,False,grassland



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,136.0,NaN,NaN,NaN,57.279412,31.686121,1.0,31.0,58.0,85.25,111.0
name,136,135,Thunderbird,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
minimum level,136.0,NaN,NaN,NaN,20.823529,14.480067,1.0,9.0,18.0,32.25,50.0
maximum level,136.0,NaN,NaN,NaN,33.036765,12.140784,10.0,22.0,38.0,45.0,50.0
fecondity,136.0,NaN,NaN,NaN,865.639706,420.953005,10.0,487.5,897.5,1240.0,1500.0
pallu refresh type,136,5,Creeps,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
night only,136,2,False,119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
refresh area,136,10,grassland,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage


Aucune valeur manquante détectée

Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 135 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • Naughty cat: 1 occurrences
    • Mian Youyou: 1 occurrences
    • green leaf rat: 1 occurrences
    • tinder fox: 1 occurrences
  - Valeur la moins fréquente: Winter Caller (1 occurrence(s))

pallu refresh type:
  - 5 valeurs uniques
  - Valeurs: ['Creeps' 'Secret Domain BOSS' 'event' 'Random dungeon boss' 'Wild BOSS']

refresh area:
  - 10 valeurs uniques
  - Valeurs: ['grassland' 'islands' 'dungeon' 'Snow' 'forest' 'volcano' 'game reserve'
 'desert' 'forest snow' 'Random events']



In [44]:
explore_dataframe(ordinary_boss, 'Ordinary Boss')


===== Exploration de Ordinary Boss =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=129, step=1)
Entries: 129 | Columns: 4


,Column,Non-Null Count,Dtype
0,name,129,object
1,HP,26,float64
2,Remote attack,18,float64
3,Riding speed (BOSS is 100 ...,129,int64


Memory usage: 12.18 KB

Dimensions: 129 lignes, 4 colonnes

Aperçu des premières lignes:


,name,HP,Remote attack,Riding speed (BOSS is 100 higher)
0,Heterogeneous Griffin BOSS,NaN,NaN,1300
1,Heterogeneous Griffin,NaN,NaN,1200
2,Ye Panda BOSS,NaN,NaN,800



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,129,119,Snow Mammoth BOSS,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HP,26.0,NaN,NaN,NaN,238.076923,121.128666,110.0,132.5,200.0,330.0,420.0
Remote attack,18.0,NaN,NaN,NaN,82.5,6.47393,75.0,75.0,82.5,90.0,90.0
Riding speed (BOSS is 100 higher),129.0,NaN,NaN,NaN,922.48062,248.665284,500.0,750.0,900.0,1100.0,1600.0



Valeurs manquantes:


,Total,Pourcentage
HP,103,79.844961
Remote attack,111,86.046512



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 119 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Snow Mammoth BOSS: 2 occurrences
    • Thunderbird: 2 occurrences
    • Thunderbird BOSS: 2 occurrences
    • snow mammoth: 2 occurrences
    • Melupa BOSS: 2 occurrences
  - Valeur la moins fréquente: Cloud sea deer (1 occurrence(s))



In [45]:
explore_dataframe(tower_boss, 'Tower Boss')


===== Exploration de Tower Boss =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=10, step=1)
Entries: 10 | Columns: 17


,Column,Non-Null Count,Dtype
0,name,10,object
1,HP,10,int64
2,melee attack,10,int64
3,remote attack,10,int64
4,defense,10,int64
5,Support,10,int64
6,experience ratio,10,int64
7,slow walking speed,10,int64
8,walking speed,10,int64
9,running speed,10,int64


Memory usage: 1.94 KB

Dimensions: 10 lignes, 17 colonnes

Aperçu des premières lignes:


,name,HP,melee attack,remote attack,defense,Support,experience ratio,slow walking speed,walking speed,running speed,riding speed,Handling speed,ignore the bluntness,ignore displacement,BiologicalGrade,endurance,fecundity
0,Victor & Heterogeneous Gri...,8000,130,200,220,90,30,80,80,850,1100,465,True,True,9,100,9999
1,Heterogeneous Griffin,120,130,120,140,90,1,80,80,850,1200,465,False,False,5,250,60
2,Zoe & Lightning Bear,6000,100,100,100,100,30,80,140,470,650,287,True,True,9,100,9999



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,10,10,Victor & Heterogeneous Gri...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HP,10.0,NaN,NaN,NaN,3303.5,3426.634282,100.0,106.25,2810.0,6375.0,8000.0
melee attack,10.0,NaN,NaN,NaN,108.0,13.165612,100.0,100.0,100.0,115.0,130.0
remote attack,10.0,NaN,NaN,NaN,123.5,29.818898,100.0,106.25,115.0,127.5,200.0
defense,10.0,NaN,NaN,NaN,121.5,36.897004,100.0,101.25,107.5,121.25,220.0
Support,10.0,NaN,NaN,NaN,96.0,5.163978,90.0,90.0,100.0,100.0,100.0
experience ratio,10.0,NaN,NaN,NaN,15.5,15.284342,1.0,1.0,15.5,30.0,30.0
slow walking speed,10.0,NaN,NaN,NaN,94.0,32.386554,60.0,80.0,80.0,100.0,150.0
walking speed,10.0,NaN,NaN,NaN,141.0,49.035135,80.0,100.0,140.0,185.0,200.0
running speed,10.0,NaN,NaN,NaN,734.0,241.348066,450.0,470.0,850.0,900.0,1000.0



Valeurs manquantes:


,Total,Pourcentage


Aucune valeur manquante détectée

Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 10 valeurs uniques
  - Valeurs: ['Victor & Heterogeneous Griffin' 'Heterogeneous Griffin'
 'Zoe & Lightning Bear' 'Lightning Bear' 'Marcus & Horus' 'Horus'
 'Lily & Lily Queen' 'lily queen' 'Axel & Pollux' 'Pollux']



In [47]:
explore_dataframe(combat_attribute, 'Combat Attribut')


===== Exploration de Combat Attribut =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=138, step=1)
Entries: 138 | Columns: 50


,Column,Non-Null Count,Dtype
0,ID,138,int64
1,Chinese name,138,object
2,Name,138,object
3,CodeName,138,object
4,OverrideNameTextID,1,object
5,NamePrefixID,0,float64
6,OverridePartnerSkillTextID,1,object
7,IsPal,138,bool
8,Tribe,138,object
9,BPClass,138,object


Memory usage: 249.28 KB

Dimensions: 138 lignes, 50 colonnes

Aperçu des premières lignes:


,ID,Chinese name,Name,CodeName,OverrideNameTextID,NamePrefixID,OverridePartnerSkillTextID,IsPal,Tribe,BPClass,variant,Volume size,rarity,Element 1,Element 2,GenusCategory,Organization,weapon,WeaponEquip,nocturnal,4D total,HP,melee attack,Remote attack,defense,support,Speed of work,Level 1,Level 20,Level 50,AIRResponse,AISightResponse,endurance,slow walking speed,walking speed,running speed,Riding sprint speed,(being) damage multiplier,catch rate,Experience multiplier,price,Must bring entry 1,Must bring entry 2,Numerical description,lv1,lv2,lv3,lv4,lv5,Skill description
0,1,Mian Youyou,Lamball,SheepBall,NaN,NaN,NaN,True,SheepBall,SheepBall,NaN,XS,1,generally,NaN,Humanoid,NaN,NaN,False,NaN,280,70,70,70,70,100,100,105-106,205-236,362-441,Friendly,NaN,100,23,40,400,NaN,2,"1,5",1,1000,NaN,NaN,Ranch skills,0,0,0,0,0,"When activated, it will tr..."
1,3,Pipi Chicken,Chikipi,ChickenPal,NaN,NaN,NaN,True,ChickenPal,ChickenPal,NaN,XS,1,generally,NaN,Bird,NaN,NaN,False,NaN,250,60,70,60,60,70,100,104-105,190-217,325-392,Friendly,NaN,100,50,50,375,NaN,2,"1,5",1,1000,NaN,NaN,Ranch skills,0,0,0,0,0,Assign it to Palu Ranch an...
2,4,green leaf rat,Lifmunk,Carbunclo,NaN,NaN,NaN,True,Carbunclo,Carbunclo,NaN,XS,1,Wood,NaN,Humanoid,NaN,NaN,False,NaN,285,75,70,70,70,100,100,105-106,205-236,362-441,Escape,NaN,100,20,40,400,NaN,1,"0,9",1,1010,NaN,NaN,Skill power,10,11,13,16,20,"After being activated, it ..."



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,138.0,NaN,NaN,NaN,57.826087,31.829361,1.0,31.25,58.5,85.75,111.0
Chinese name,138,137,Thunderbird,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,138,138,Lamball,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CodeName,138,138,SheepBall,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverrideNameTextID,1,1,PAL_NAME_PlantSlime,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NamePrefixID,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverridePartnerSkillTextID,1,1,PARTNERSKILL_PlantSlime,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IsPal,138,1,True,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tribe,138,137,PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BPClass,138,138,SheepBall,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
OverrideNameTextID,137,99.275362
NamePrefixID,138,100.000000
OverridePartnerSkillTextID,137,99.275362
variant,105,76.086957
Element 2,112,81.159420
Organization,138,100.000000
weapon,138,100.000000
nocturnal,113,81.884058
AISightResponse,138,100.000000
Riding sprint speed,77,55.797101



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

Chinese name:
  - 137 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • Pipi Chicken: 1 occurrences
    • green leaf rat: 1 occurrences
    • tinder fox: 1 occurrences
    • surf duck: 1 occurrences
  - Valeur la moins fréquente: Vortex Dragon (1 occurrence(s))

Name:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Lamball: 1 occurrences
    • Chikipi: 1 occurrences
    • Lifmunk: 1 occurrences
    • Foxparks: 1 occurrences
    • Fuack: 1 occurrences
  - Valeur la moins fréquente: Jetragon (1 occurrence(s))

CodeName:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • SheepBall: 1 occurrences
    • ChickenPal: 1 occurrences
    • Carbunclo: 1 occurrences
    • Kitsunebi: 1 occurrences
    • BluePlatypus: 1 occurrences
  - Valeur la moins fréquente: JetDragon (1 occurrence(s))

OverrideNameTextID:
  - 2 valeurs uniques
  - V

In [48]:
explore_dataframe(hidden_attribute, 'Hidden Attribute')


===== Exploration de Hidden Attribute =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=175, step=1)
Entries: 175 | Columns: 70


,Column,Non-Null Count,Dtype
0,name,175,object
1,OverrideNameTextID,160,object
2,NamePrefixID,160,object
3,OverridePartnerSkillTextID,160,object
4,IsPal,175,bool
...,...,...,...
65,pasture,175,int64
66,Passive skill 1,17,object
67,Passive skill 2,4,object
68,Passive skill 3,0,float64


Memory usage: 227.22 KB

Dimensions: 175 lignes, 70 colonnes

Aperçu des premières lignes:


,name,OverrideNameTextID,NamePrefixID,OverridePartnerSkillTextID,IsPal,Tribe,BPClass,Pictorial ID,ZukanIndexSuffix,Size,rarity,Element 1,Element 2,GenusCategory,Organization,weapon,WeaponEquip,HP,melee attack,Remote attack,defense,support,CraftSpeed,(being) damage multiplier,Capture probability,Experience multiplier,price,AIRResponse,AISightResponse,slow walking speed,walking speed,running speed,Riding sprint speed,Handling speed,IsBoss,IsTowerBoss,BattleBGM,IgnoreLeanBack,IgnoreBlowAway,MaxFullStomach,FullStomachDecreaseRate,FoodAmount,ViewingDistance,ViewingAngle,HearingRate,NooseTrap,Nocturnal,BiologicalGrade,Predator,Edible,endurance,Male probability,fecundity,Breathing fire,watering,planting,generate electricity,manual,collection,logging,Mining,OilExtraction (not shown in game),pharmaceutical,cool down,carry,pasture,Passive skill 1,Passive skill 2,Passive skill 3,Passive skill 4
0,BOSS_Alpaca,Alpaca,BOSS_NAME_Alpaca,PARTNERSKILL_Alpaca,True,Alpaca,BOSS_Alpaca,-1,NaN,XL,3,Normal,NaN,FourLegged,NaN,NaN,False,90,90,90,90,100,100,"0,4","0,7",10,3040,NotInterested,NaN,62,62,460,900,261,True,False,FieldBoss,False,False,225,1,3,25,90,1,False,False,9,False,True,100,50,890,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN
1,BOSS_AmaterasuWolf,AmaterasuWolf,BOSS_NAME_AmaterasuWolf,PARTNERSKILL_AmaterasuWolf,True,AmaterasuWolf,BOSS_AmaterasuWolf,-1,NaN,XL,6,Fire,NaN,FourLegged,NaN,NaN,False,100,70,115,100,110,100,"0,2","0,7",10,3170,Boss,NaN,80,130,700,1000,415,True,False,FieldBoss,False,False,300,1,4,25,90,1,False,False,9,True,True,100,50,830,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,BOSS_BadCatgirl,BadCatgirl,BOSS_NAME_BadCatgirl,PARTNERSKILL_BadCatgirl,True,BadCatgirl,BOSS_BadCatgirl,-1,NaN,XL,0,Dark,NaN,Humanoid,NaN,NaN,False,100,100,100,100,100,100,"0,2","0,7",10,4690,Boss,NaN,-1,-1,-1,-1,-1,True,False,FieldBoss,False,False,225,1,3,25,90,1,False,True,9,False,True,100,50,0,0,0,0,0,2,3,1,0,0,0,0,2,0,NaN,NaN,NaN,NaN



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,175,175,BOSS_Alpaca,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverrideNameTextID,160,159,PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NamePrefixID,160,159,BOSS_NAME_PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverridePartnerSkillTextID,160,159,PARTNERSKILL_PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IsPal,175,1,True,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
pasture,175.0,NaN,NaN,NaN,0.074286,0.262988,0.0,0.0,0.0,0.0,1.0
Passive skill 1,17,11,Legend,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Passive skill 2,4,4,ElementBoost_Dark_2_PAL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Passive skill 3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
OverrideNameTextID,15,8.571429
NamePrefixID,15,8.571429
OverridePartnerSkillTextID,15,8.571429
ZukanIndexSuffix,175,100.000000
Element 2,145,82.857143
Organization,175,100.000000
weapon,175,100.000000
AISightResponse,175,100.000000
Passive skill 1,158,90.285714
Passive skill 2,171,97.714286



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 175 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • BOSS_Alpaca: 1 occurrences
    • BOSS_AmaterasuWolf: 1 occurrences
    • BOSS_BadCatgirl: 1 occurrences
    • BOSS_Baphomet: 1 occurrences
    • BOSS_Baphomet_Dark: 1 occurrences
  - Valeur la moins fréquente: BOSS_ThunderBird (1 occurrence(s))

OverrideNameTextID:
  - 160 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • PlantSlime: 2 occurrences
    • AmaterasuWolf: 1 occurrences
    • Alpaca: 1 occurrences
    • Baphomet: 1 occurrences
    • Baphomet_Dark: 1 occurrences
  - Valeur la moins fréquente: ThunderBird (1 occurrence(s))

NamePrefixID:
  - 160 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • BOSS_NAME_PlantSlime: 2 occurrences
    • BOSS_NAME_AmaterasuWolf: 1 occurrences
    • BOSS_NAME_Alpaca: 1 occurrences
    • BOSS_NAME_Baphomet: 1 occurrences
    • BOSS_NAME_Baphomet_Dark: 1 occurrences
  - V